In [1]:
# Library Imports
import pandas as pd
import numpy as np
import math

In [2]:
dfBSC = pd.read_csv('Data/evodefi.csv')

# Convert to readable timestamps
dfBSC['timeGMT'] = pd.to_datetime(dfBSC['timeStamp'], unit='s')
dfBSC.head(2)

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations,timeGMT
0,7929705,1622588943,0xff2842f9b1b5687c3c830b18e98245f2f797df8d5af6...,115,0xad5deb5806d95c4197eb4158c3375f74d013c5c8bf7d...,0x5b646303cc2a1dd58477284c57915a33136e67bc,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,0x3247554b2bad31d5120b98d5f51df5a406d6b524,324934794118733645,Binance-Peg USD Coin,USDC,18,126,291743,5000000000,276743,18740884,deprecated,10049,2021-06-01 23:09:03
1,7929705,1622588943,0xff2842f9b1b5687c3c830b18e98245f2f797df8d5af6...,115,0xad5deb5806d95c4197eb4158c3375f74d013c5c8bf7d...,0x5b646303cc2a1dd58477284c57915a33136e67bc,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,0x3247554b2bad31d5120b98d5f51df5a406d6b524,64662024029627995417,Binance-Peg USD Coin,USDC,18,126,291743,5000000000,276743,18740884,deprecated,10049,2021-06-01 23:09:03


In [3]:
dfPolygon = pd.read_csv('Data/evoDeFiPolygon.csv')

# Convert to readable timestamps
dfPolygon['Block_Time'] = dfPolygon['Block_Time'].str.replace('T', ' ')
dfPolygon['Block_Time'] = dfPolygon['Block_Time'].str.replace('Z', '')
dfPolygon['Block_Time'] = pd.to_datetime(dfPolygon['Block_Time'])
dfPolygon.head(2)

,TxHash,Block_Time,Block_Height,Gas,User_Address,From,To,Token_Contract,Value
0,0x75258bedda370511ab5b448a6ade62d15c2f597deaa4...,2021-06-03 14:28:13,15286504,87349,0x598e630ea3e4c3de045ca0dd5f377565ec2f540c,0xbca17c85f7e965ae6f5d05769d300e59221cf1e2,0x1e6b5986852ad914c5f3a7d3fa19478d6371053e,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,2506787369
1,0xf182b012a5309df7193e25e5415661b355de7282d587...,2021-06-03 14:27:53,15286494,104437,0x598e630ea3e4c3de045ca0dd5f377565ec2f540c,0xbca17c85f7e965ae6f5d05769d300e59221cf1e2,0x5dc4fbdc9f05ea92423d5484eb84c004200824e1,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,1008609360


In [4]:
dfPolygon.Token_Contract.value_counts()

0x2791bca1f2de4661ed88a30c99a7a9449aa84174    7191
0x7ceb23fd6bc0add59e62ac25578270cff1b9f619     510
0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270     284
0x161c0ece60dcfcdc3e4bdd5f1cde3ed2f68285a9      25
Name: Token_Contract, dtype: int64

In [5]:
# Add token details
def label_token(row):
    if(row['Token_Contract'] == "0x2791bca1f2de4661ed88a30c99a7a9449aa84174"):
        return(["USDC", "USD Coin (PoS)", "18"])
    if(row['Token_Contract'] == "0x7ceb23fd6bc0add59e62ac25578270cff1b9f619"):
        return(["WETH", "Wrapped Ether", "18"])
    if(row['Token_Contract'] == "0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270"):
        return(["WMATIC", "Wrapped Matic", "18"])
    if(row['Token_Contract'] == "0x161c0ece60dcfcdc3e4bdd5f1cde3ed2f68285a9"):
        return(["GEN", "Gen Token", "18"])
    return("Undefined")

dfPolygon['Token_Symbol'] = dfPolygon.apply(lambda row: (label_token(row)[0]), axis=1)
dfPolygon['Token_Name'] = dfPolygon.apply(lambda row: (label_token(row)[1]), axis=1)
dfPolygon['Token_Decimals'] = dfPolygon.apply(lambda row: (label_token(row)[2]), axis=1)
dfPolygon.head(4)

,TxHash,Block_Time,Block_Height,Gas,User_Address,From,To,Token_Contract,Value,Token_Symbol,Token_Name,Token_Decimals
0,0x75258bedda370511ab5b448a6ade62d15c2f597deaa4...,2021-06-03 14:28:13,15286504,87349,0x598e630ea3e4c3de045ca0dd5f377565ec2f540c,0xbca17c85f7e965ae6f5d05769d300e59221cf1e2,0x1e6b5986852ad914c5f3a7d3fa19478d6371053e,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,2506787369,USDC,USD Coin (PoS),18
1,0xf182b012a5309df7193e25e5415661b355de7282d587...,2021-06-03 14:27:53,15286494,104437,0x598e630ea3e4c3de045ca0dd5f377565ec2f540c,0xbca17c85f7e965ae6f5d05769d300e59221cf1e2,0x5dc4fbdc9f05ea92423d5484eb84c004200824e1,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,1008609360,USDC,USD Coin (PoS),18
2,0xaf4e53e77ac86099a451923651be0943ac984d747a38...,2021-06-03 14:26:35,15286457,104461,0x598e630ea3e4c3de045ca0dd5f377565ec2f540c,0xbca17c85f7e965ae6f5d05769d300e59221cf1e2,0x0dd2171dd7a543cb3983af30a5e5b179182be9dc,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,6086107541,USDC,USD Coin (PoS),18
3,0xebc6982ff773efb69bb940a5b71a85eee2a3b2f0e9d3...,2021-06-03 14:26:19,15286449,104425,0x598e630ea3e4c3de045ca0dd5f377565ec2f540c,0xbca17c85f7e965ae6f5d05769d300e59221cf1e2,0x8f25098cdd6e815d2fc138516fd3da75f161893b,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,49750000,USDC,USD Coin (PoS),18


In [6]:
# Fix token value details
def fix_value(row):
    if(row['Token_Symbol'] == "USDC"):
        return(int(row["Value"]) * pow(10, -6))
    if(row['Token_Symbol'] == "WETH"):
        return(int(row["Value"]) * pow(10, -18))
    if(row['Token_Symbol'] == "WMATIC"):
        return(int(row["Value"]) * pow(10, -18))
    if(row['Token_Symbol'] == "GEN"):
        return(int(row["Value"]) * pow(10, -18))
    return(0)

dfPolygon['Value'] = dfPolygon.apply(lambda row: fix_value(row), axis=1)
dfPolygon.head(2)

,TxHash,Block_Time,Block_Height,Gas,User_Address,From,To,Token_Contract,Value,Token_Symbol,Token_Name,Token_Decimals
0,0x75258bedda370511ab5b448a6ade62d15c2f597deaa4...,2021-06-03 14:28:13,15286504,87349,0x598e630ea3e4c3de045ca0dd5f377565ec2f540c,0xbca17c85f7e965ae6f5d05769d300e59221cf1e2,0x1e6b5986852ad914c5f3a7d3fa19478d6371053e,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,2506.787369,USDC,USD Coin (PoS),18
1,0xf182b012a5309df7193e25e5415661b355de7282d587...,2021-06-03 14:27:53,15286494,104437,0x598e630ea3e4c3de045ca0dd5f377565ec2f540c,0xbca17c85f7e965ae6f5d05769d300e59221cf1e2,0x5dc4fbdc9f05ea92423d5484eb84c004200824e1,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,1008.609360,USDC,USD Coin (PoS),18


In [16]:
dfPolygonIn = dfPolygon[dfPolygon['From']=="0xbca17c85f7e965ae6f5d05769d300e59221cf1e2"]
print("The dataframe has {} rows and {} columns.".format(dfPolygonIn.shape[0], dfPolygonIn.shape[1]))

The dataframe has 7915 rows and 12 columns.
